In [1]:
%load_ext sql
%sql sqlite:///chinook.db

## Задание 1
Вывести названия всех групп, их песен и названия их альбомов для всех треков жанра Rock, приобретенные сотрудниками Microsoft.


In [4]:
%%sql
select ar.Name as Artist, al.Title as Album, t.Name as Track
    from invoice_items ii
    join tracks t
        on ii.TrackId = t.TrackId
    join genres g
        on t.GenreId = g.GenreId
    join albums al
        on al.AlbumId = t.AlbumId
    join artists ar
        on al.ArtistId = ar.ArtistId
    join invoices i
        on ii.InvoiceId = i.InvoiceId
    join Customers c
        on i.CustomerId = c.CustomerId
where c.Company LIKE "Microsoft%" AND g.Name LIKE "Rock";

 * sqlite:///chinook.db
Done.


Artist,Album,Track
Guns N' Roses,Use Your Illusion I,Double Talkin' Jive
Guns N' Roses,Use Your Illusion I,The Garden
Guns N' Roses,Use Your Illusion I,Don't Damn Me
Guns N' Roses,Use Your Illusion I,Dead Horse
Foo Fighters,In Your Honor [Disc 1],No Way Back
Foo Fighters,In Your Honor [Disc 2],Still
Foo Fighters,In Your Honor [Disc 2],Razor
Foo Fighters,The Colour And The Shape,My Hero


## Задание 2
Вывести 10 самых больших по размеру треков жанра Blues и формата MPEG (используйте команду LIMIT 10). Только название и автор.


In [5]:
%%sql
select ar.Name as Artist, t.Name as Track
    from tracks t
    join genres g
        on t.GenreId = g.GenreId
    join media_types mt
        on t.MediaTypeId = mt.MediaTypeId
    join albums al
        on al.AlbumId = t.AlbumId
    join artists ar
        on al.ArtistId = ar.ArtistId
where g.Name LIKE "Blues" AND mt.Name LIKE "MPEG%"
order by t.Bytes desc
limit 10;

 * sqlite:///chinook.db
Done.


Artist,Track
Buddy Guy,Talkin' 'Bout Women Obviously
Stevie Ray Vaughan & Double Trouble,Riviera Paradise
The Black Crowes,Title Song
Eric Clapton,Old Love
The Black Crowes,Wiser Time
Buddy Guy,Stone Crazy
Eric Clapton,Layla
The Black Crowes,Girl From A Pawnshop
The Black Crowes,Sometimes Salvation
Eric Clapton,Running On Faith


## Задание 3
Для каждой компании вывести общее количество купленных джазовых композиций.


In [13]:
%%sql
select c.Company as Company, sum(ii.Quantity) as Bought_Tracks
    from customers c
    join invoices i
        on c.CustomerId = i.CustomerId
    join invoice_items ii
        on i.InvoiceId = ii.InvoiceId
    join tracks t
        on ii.TrackId = t.TrackId
    join genres g
        on g.GenreId = t.GenreId
where g.Name like "Jazz" and c.Company is not null
group by c.Company;


 * sqlite:///chinook.db
Done.


Company,Bought_Tracks
Apple Inc.,2
Google Inc.,2
JetBrains s.r.o.,3
Microsoft Corporation,2
Telus,2


## Задание 4
Вывести топ 3 альбома по продолжительности (продолжительность альбома - сумма длин треков в нём). Только название и продолжительность.


In [42]:
%%sql

select al.Title as Album, strftime('%H:%M:%S', SUM(t.Milliseconds) / 1000, 'unixepoch') AS Time
    from tracks t
    join albums al
        on al.AlbumId = t.AlbumId
group by al.Title
order by sum(t.Milliseconds) desc
limit 3;

 * sqlite:///chinook.db
Done.


Album,Time
"Lost, Season 3",19:37:45
"Battlestar Galactica (Classic), Season 1",19:30:13
"Lost, Season 1",18:00:54


## Задание 5
Для каждого набора (жанр, тип медиа) вывести среднюю цену по стоимости трека и их общее количество, причем вывести только те наборы, для которых **все** треки стоят больше 1,5$ и для которых в наборе есть хотя бы один трек.

In [56]:
%%sql

select distinct g.Name as Genre, mt.Name as Media_Type, avg(t.UnitPrice) as Average_Price, count(t.TrackId) as Total_Tracks
    from genres g
    join tracks t
        on t.GenreId = g.GenreId
    join media_types mt
        on mt.MediaTypeId = t.MediaTypeId
group by g.Name, mt.Name
having sum(t.UnitPrice) > 1.5
limit 1000;

 * sqlite:///chinook.db
Done.


Genre,Media_Type,Average_Price,Total_Tracks
Alternative,Protected AAC audio file,0.9899999999999998,38
Alternative & Punk,MPEG audio file,0.9900000000000047,332
Blues,MPEG audio file,0.9899999999999994,81
Bossa Nova,MPEG audio file,0.9900000000000001,15
Classical,Protected AAC audio file,0.9900000000000004,67
Classical,Purchased AAC audio file,0.9900000000000001,6
Comedy,Protected MPEG-4 video file,1.9899999999999995,17
Drama,Protected MPEG-4 video file,1.9899999999999978,64
Easy Listening,MPEG audio file,0.9899999999999997,24
Electronica/Dance,AAC audio file,0.99,2
